## Loading dataset 

In [3]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [4]:
! kaggle datasets download -d rahilmehtaucoe2784/fashion

100% 191M/191M [00:01<00:00, 162MB/s]



In [5]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

## installing and importing Libraries 

In [6]:
!pip install -q tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 616 kB 8.1 MB/s 
     |████████████████████████████████| 1.2 MB 57.6 MB/s 
     |████████████████████████████████| 1.1 MB 53.9 MB/s 
     |████████████████████████████████| 120 kB 61.2 MB/s 
     |████████████████████████████████| 840 kB 38.9 MB/s 
     |████████████████████████████████| 3.4 MB 57.4 MB/s 
     |████████████████████████████████| 6.4 MB 53.9 MB/s 
     |████████████████████████████████| 1.1 MB 52.0 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 596 kB 56.7 MB/s 
     |████████████████████████████████| 234 kB 54.8 MB/s 
     |████████████████████████████████| 87 kB 6.9 MB/s 
     |████████████████████████████████| 25.3 MB 1.4 MB/s 
     |████████████████████████████████| 99 kB 9.1 MB/s 
     |████████████████████████████████| 352 kB 46.7 MB/s 
     |████████████████████████████████| 47.7 MB 1.2 MB/s 
     |████████████████████████████████| 462 kB 55.6 MB/s 
     |███████████████

In [7]:
import numpy as np
import os

#from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

## Training 

In [8]:
train_image_dir = '/content/Final Img copy 2/train'
train_annotation_dir = '/content/Final Img copy 2/train anotation'
validate_image_dir = '/content/Final Img copy 2/valid'
validate_anaotation_dir = '/content/Final Img copy 2/valid anotation '

In [9]:

spec = model_spec.get('efficientdet_lite1')

In [10]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani", 7:"tshirt"})

In [11]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani",7:"tshirt"})

In [12]:
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=10, 
                               batch_size=5, 
                               train_whole_model=True, 
                               validation_data=validate_dir
                               )

Epoch 1/10
176/176 [==============================] - 115s 347ms/step - det_loss: 1.2121 - cls_loss: 0.9307 - box_loss: 0.0056 - reg_l2_loss: 0.0715 - loss: 1.2837 - learning_rate: 0.0071 - gradient_norm: 3.0839 - val_det_loss: 0.7549 - val_cls_loss: 0.6780 - val_box_loss: 0.0015 - val_reg_l2_loss: 0.0716 - val_loss: 0.8265
Epoch 2/10
176/176 [==============================] - 58s 332ms/step - det_loss: 0.6801 - cls_loss: 0.5731 - box_loss: 0.0021 - reg_l2_loss: 0.0717 - loss: 0.7519 - learning_rate: 0.0058 - gradient_norm: 3.5092 - val_det_loss: 0.6244 - val_cls_loss: 0.5862 - val_box_loss: 7.6419e-04 - val_reg_l2_loss: 0.0718 - val_loss: 0.6962
Epoch 3/10
176/176 [==============================] - 58s 331ms/step - det_loss: 0.5328 - cls_loss: 0.4613 - box_loss: 0.0014 - reg_l2_loss: 0.0719 - loss: 0.6046 - learning_rate: 0.0051 - gradient_norm: 3.4071 - val_det_loss: 0.4961 - val_cls_loss: 0.4715 - val_box_loss: 4.9329e-04 - val_reg_l2_loss: 0.0719 - val_loss: 0.5681
Epoch 4/10
176/1

## Evaluating the model 

In [13]:
model.evaluate(validate_dir,batch_size=5)

49/49 [==============================] - 17s 244ms/step



{'AP': 0.5740849,
 'AP50': 0.6696043,
 'AP75': 0.6460294,
 'AP_/Casual Shirt': 0.49701744,
 'AP_/Desgin sherwani': 0.74189126,
 'AP_/Design Kurta': 0.43073243,
 'AP_/Formal Shirt': 0.6729827,
 'AP_/Plain kurta': 0.2559933,
 'AP_/Plain sherwani': 0.6243117,
 'AP_/tshirt': 0.7956652,
 'APl': 0.5741126,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.8674846,
 'ARm': -1.0,
 'ARmax1': 0.8388121,
 'ARmax10': 0.8674846,
 'ARmax100': 0.8674846,
 'ARs': -1.0}

## Saving Model 

In [14]:
model.export(export_dir='/content/')